In [1]:
import pandas as pd

from api.data import (store_database_for_eys_gene,
                      parse_lovd,
                      parse_gnomad,
                      LOVD_PATH,
                      set_lovd_dtypes,
                      set_gnomad_dtypes,
                      download_data_from_gnomad_eys,
                      merge_gnomad_lovd,
                      GNOMAD_PATH,
                      )
from api.data import save_lovd_as_vcf


pd.options.display.max_columns = 0

In [2]:
store_database_for_eys_gene("lovd", override=False)

The file at ../data/lovd/lovd_data.txt already exists.


In [3]:
download_path = 'C:\\Users\\Kajus\\Desktop\\gnomad_data_downloaded.csv'
data = parse_lovd(LOVD_PATH + "/lovd_data.txt")

In [4]:
gnomad_data = download_data_from_gnomad_eys(path=download_path, override=False)

display(gnomad_data)

,Popmax,Popmax population,Homozygote Count,Allele Frequency,variant_id
0,0.000016,African/African American,0.0,1.807419e-06,6-63720525-A-G
1,0.000192,East Asian,0.0,6.573844e-06,6-63720525-A-T
2,0.000000,,0.0,0.000000e+00,6-63720525-A-C
3,0.000020,South Asian,0.0,1.045299e-06,6-63720526-T-A
4,0.000000,,0.0,0.000000e+00,6-63720527-G-T
...,...,...,...,...,...
14295,0.000000,,0.0,0.000000e+00,6-65495479-G-T
14296,0.000031,African/African American,0.0,1.446349e-06,6-65495479-G-A
14297,0.000070,Admixed American,0.0,2.629510e-06,6-65495482-A-G
14298,0.000060,South Asian,0.0,3.645085e-06,6-65495484-T-G


In [5]:
store_database_for_eys_gene('gnomad', False)

gnomad_data_2 = parse_gnomad(GNOMAD_PATH +'/gnomad_data.csv')

In [6]:
display(gnomad_data_2)

,gnomAD ID,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,HGVS Consequence,Protein Consequence,Transcript Consequence,VEP Annotation,ClinVar Clinical Significance,ClinVar Variation ID,Flags,Allele Count,Allele Number,Allele Frequency,Homozygote Count,Hemizygote Count,Filters - joint,GroupMax FAF group,GroupMax FAF frequency,cadd,revel_max,spliceai_ds_max,pangolin_largest_ds,phylop,sift_max,polyphen_max,Allele Count African/African American,Allele Number African/African American,Homozygote Count African/African American,Hemizygote Count African/African American,Allele Count Admixed American,Allele Number Admixed American,Homozygote Count Admixed American,Hemizygote Count Admixed American,Allele Count Ashkenazi Jewish,Allele Number Ashkenazi Jewish,Homozygote Count Ashkenazi Jewish,Hemizygote Count Ashkenazi Jewish,Allele Count East Asian,Allele Number East Asian,Homozygote Count East Asian,Hemizygote Count East Asian,Allele Count European (Finnish),Allele Number European (Finnish),Homozygote Count European (Finnish),Hemizygote Count European (Finnish),Allele Count Middle Eastern,Allele Number Middle Eastern,Homozygote Count Middle Eastern,Hemizygote Count Middle Eastern,Allele Count European (non-Finnish),Allele Number European (non-Finnish),Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Allele Count Amish,Allele Number Amish,Homozygote Count Amish,Hemizygote Count Amish,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian,Allele Count Remaining,Allele Number Remaining,Homozygote Count Remaining,Hemizygote Count Remaining
0,6-63720525-A-G,6,63720525,rs1768331164,A,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000503581.6,c.*71T>C,NaN,c.*71T>C,3_prime_UTR_variant,NaN,NaN,NaN,2,1106550,1.807419e-06,0,0,PASS,NaN,NaN,0.238,NaN,0.00,0.00,-2.180,NaN,NaN,1,62852,0,0,0,31732,0,0,0,20048,0,0,0,37994,0,0,0,54334,0,0,0,3468,0,0,1,795766,0,0,0,912,0,0,0,55362,0,0,0,44082,0,0
1,6-63720525-A-T,6,63720525,rs1768331164,A,T,gnomAD Genomes,NaN,PASS,ENST00000503581.6,c.*71T>A,NaN,c.*71T>A,3_prime_UTR_variant,NaN,NaN,NaN,1,1106550,9.037097e-07,0,0,PASS,NaN,NaN,0.193,NaN,0.00,0.00,-2.180,NaN,NaN,0,62852,0,0,0,31732,0,0,0,20048,0,0,1,37994,0,0,0,54334,0,0,0,3468,0,0,0,795766,0,0,0,912,0,0,0,55362,0,0,0,44082,0,0
2,6-63720526-T-A,6,63720526,NaN,T,A,gnomAD Exomes,PASS,NaN,ENST00000503581.6,c.*70A>T,NaN,c.*70A>T,3_prime_UTR_variant,NaN,NaN,NaN,1,1108894,9.017995e-07,0,0,PASS,NaN,NaN,1.430,NaN,0.00,0.00,0.276,NaN,NaN,0,62994,0,0,0,31764,0,0,0,20076,0,0,0,37960,0,0,0,54376,0,0,0,3462,0,0,0,797828,0,0,0,912,0,0,1,55360,0,0,0,44162,0,0
3,6-63720531-C-CAA,6,63720531,NaN,C,CAA,gnomAD Exomes,PASS,NaN,ENST00000503581.6,c.*63_*64dup,NaN,c.*63_*64dup,3_prime_UTR_variant,NaN,NaN,NaN,3,1214050,2.471068e-06,0,0,PASS,nfe,3.700000e-07,4.830,NaN,0.00,0.00,0.460,NaN,NaN,0,64966,0,0,0,32056,0,0,0,20754,0,0,0,38418,0,0,0,54662,0,0,0,3736,0,0,2,893326,0,0,0,912,0,0,0,57520,0,0,1,47700,0,0
4,6-63720531-C-G,6,63720531,rs927390284,C,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000503581.6,c.*65G>C,NaN,c.*65G>C,3_prime_UTR_variant,NaN,NaN,NaN,5,1213934,4.118840e-06,0,0,PASS,afr,5.120000e-06,5.760,NaN,0.00,0.00,0.460,NaN,NaN,2,64844,0,0,0,32038,0,0,0,20754,0,0,0,38430,0,0,0,54662,0,0,0,3758,0,0,3,893334,0,0,0,912,0,0,0,57524,0,0,0,47678,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11083,6-65495478-G-A,6,65495478,rs530118054,G,A,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000503581.6,c.-68C>T,NaN,c.-68C>T,5_prime_UTR_variant,Uncertain significance,357751.0,NaN,142,1536992,9.238825e-05,0,0,PASS,nfe,8.983000e-05,5.460,NaN,0.00,-0.11,0.424,NaN,NaN,4,73554,0,0,0,57632,0,0,0,28976,0,0,4,4

In [7]:
gnomad_data.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_api.csv', index=False)

In [8]:
len(gnomad_data_2), len(gnomad_data)

print(len(gnomad_data_2) - len(gnomad_data))

-3212


In [11]:
gnomad_data

,Popmax,Popmax population,Homozygote Count,Allele Frequency,variant_id
0,0.000016,African/African American,0.0,1.807419e-06,6-63720525-A-G
1,0.000192,East Asian,0.0,6.573844e-06,6-63720525-A-T
2,0.000000,,0.0,0.000000e+00,6-63720525-A-C
3,0.000020,South Asian,0.0,1.045299e-06,6-63720526-T-A
4,0.000000,,0.0,0.000000e+00,6-63720527-G-T
...,...,...,...,...,...
14295,0.000000,,0.0,0.000000e+00,6-65495479-G-T
14296,0.000031,African/African American,0.0,1.446349e-06,6-65495479-G-A
14297,0.000070,Admixed American,0.0,2.629510e-06,6-65495482-A-G
14298,0.000060,South Asian,0.0,3.645085e-06,6-65495484-T-G


In [12]:
missing_from_api = []

for i in gnomad_data['variant_id']:
    if(i in gnomad_data_2['gnomAD ID'].values):
        continue
    missing_from_api.append(i)

len(missing_from_api)

missing_data = gnomad_data.loc[gnomad_data['variant_id'].isin(missing_from_api)]

missing_data

,Popmax,Popmax population,Homozygote Count,Allele Frequency,variant_id
2,0.0,,0.0,0.0,6-63720525-A-C
4,0.0,,0.0,0.0,6-63720527-G-T
5,0.0,,0.0,0.0,6-63720527-G-A
8,0.0,,0.0,0.0,6-63720531-C-A
14,0.0,,0.0,0.0,6-63720535-G-T
...,...,...,...,...,...
14279,0.0,,0.0,0.0,6-65495463-G-T
14287,0.0,,0.0,0.0,6-65495471-T-C
14291,0.0,,0.0,0.0,6-65495477-C-A
14292,0.0,,0.0,0.0,6-65495477-C-CACAACTTTACTT


In [ ]:
missing_data.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_missing.csv', index=False)

In [ ]:
set_lovd_dtypes(data)
set_gnomad_dtypes(gnomad_data)

variants_on_genome = data["Variants_On_Genome"].copy()

lovd_data = pd.merge(data["Variants_On_Transcripts"],
                       variants_on_genome[['id','VariantOnGenome/DNA/hg38']],
                       on='id',
                       how='left')

gnomad_data = gnomad_data.copy()
final_data = merge_gnomad_lovd(lovd_data, gnomad_data)
final_data

In [ ]:
for i in data:
    print(i)
    display(data[i])

In [ ]:
set_lovd_dtypes(data)
for i in data:
    print(i)
    display(data[i].info())

In [ ]:
save_lovd_as_vcf(data["Variants_On_Genome"], "./lovd.vcf")

In [ ]:
from subprocess import Popen

process = Popen("spliceai -I ./lovd.vcf -O ./lovd_output.vcf -R ../tools/spliceai/hg38.fa -A grch38".split())
process.wait()

In [ ]:
from api.tools import get_revel_scores

chromosome = 6
position = 65655758

results = get_revel_scores(chromosome, position)

display(results)

In [ ]:


chromosome = 6
position = 65655758

lovd_data = pd.merge(lovd_data["Variants_On_Transcripts"],
                       variants_on_genome[['id','VariantOnGenome/DNA','VariantOnGenome/DNA/hg38']],
                       on='id',
                       how='left')

display(results)

In [1]:
from tests.tools.cadd import add_cadd_eval_column
from app.back_end.data import (download_selected_database_for_eys_gene,
                               merge_gnomad_lovd, parse_gnomad, set_gnomad_dtypes,
                               parse_lovd,set_lovd_dtypes,LOVD_PATH, GNOMAD_PATH)
import pandas as pd

download_selected_database_for_eys_gene("lovd", save_path="../data/lovd/lovd_data_text.txt", override=False)
download_selected_database_for_eys_gene("gnomad", save_path="../data/gnomad/gnomad_data_text.csv", override=False)

lovd_data = parse_lovd(LOVD_PATH + "lovd_data_text.txt")
gnomad_data = parse_gnomad(GNOMAD_PATH+'gnomad_data_text.csv')

set_lovd_dtypes(lovd_data)
set_gnomad_dtypes(gnomad_data)

variants_on_genome = lovd_data["Variants_On_Genome"].copy()

lovd_data = pd.merge(lovd_data["Variants_On_Transcripts"],
                       variants_on_genome[['id','VariantOnGenome/DNA','VariantOnGenome/DNA/hg38','chromosome','position_g_start','position_g_end']],
                       on='id',
                       how='left')

gnomad_data = gnomad_data.copy()
data = merge_gnomad_lovd(lovd_data, gnomad_data[["variant_id"]])
data

The file at ../data/lovd/lovd_data_text.txt already exists.
The file at ../data/gnomad/gnomad_data_text.csv already exists.


,id,transcriptid,effectid,position_c_start,position_c_start_intron,position_c_end,position_c_end_intron,VariantOnTranscript/DNA,VariantOnTranscript/RNA,VariantOnTranscript/Protein,VariantOnTranscript/Exon,VariantOnGenome/DNA,VariantOnGenome/DNA/hg38,chromosome,position_g_start,position_g_end,hg38_gnomad_format,variant_id_gnomad
0,822823,7329,70,632,0,632,0,c.632G>A,r.(?),p.(Cys211Tyr),,g.216052273del,g.215878931del,6,216052273,216052273,6-215878931-del,NaN
1,822787,7329,70,8391,0,8391,0,c.8391del,r.(?),p.(Gly2799Valfs*31),,g.216052410C>T,g.215879068C>T,6,216052410,216052410,6-215879068-C-T,NaN
2,822843,7329,70,5608,0,5608,0,c.5608C>T,r.(?),p.(Arg1870Trp),,g.216246607G>A,g.216073265G>A,6,216246607,216246607,6-216073265-G-A,NaN
3,822771,7329,70,8206,0,8206,0,c.8206G>C,r.(?),p.(Ala2736Pro),,g.216500979C>T,g.216327637C>T,6,216500979,216500979,6-216327637-C-T,NaN
4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,6-63720525-A-C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16409,959060,7329,70,9383,0,9387,0,c.9383_9387del,r.(?),p.(Lys3128ArgfsTer7),,g.64430545_64430549del,g.63720649_63720653del,6,64430545,64430549,?,NaN
16410,959064,7329,50,0,0,0,0,c.-538_862+10652{1}inv,r.?,p.?,_1_5i,g.66189835_67841160inv,g.65479942_67131267inv,6,66189835,67841160,?,NaN
16411,985494,7329,70,2137,20590,3444,-29847,c.2137+20590_3444-29847del,r.?,p.(Val713AspfsTer14),,g.65365985_65746917del,g.64656092_65037024del,6,65365985,65746917,?,NaN
16412,986425,7329,90,4361,0,4362,0,c.4361_4362delinsAG,r.(?),p.(Ser1454Ter),26,g.65301398_65301399delinsCT,g.64591505_64591506delinsCT,6,65301398,65301399,?,NaN
